# Chessboard detection

In [ ]:
import imageio, tqdm
import numpy as np
import joblib
import matplotlib.pyplot as plt
import cv2, os, time, glob
import networkx as nx
na = np.newaxis

def orient_board(uvs, markers, board=(7,10)):
    """flip order of chessboard points so that first is farthest from markers"""
    
    distance = np.sqrt(((uvs - markers.mean(1)[:,na,:])**2).sum(-1))
    flip = distance[:,0] < distance[:,-1]
    
    cols,rows = board
    uvs[flip] = uvs[flip].reshape(-1,rows,cols,2)[:,::-1].reshape(-1,rows*cols,2)   
    return uvs

board = (7,10)

prefixes = []
img_sizes = []
all_chessboard_uvs = []

# load and orient chessboard detections
calib_dir = '../../data/23_3_16_calibration/'
for path in tqdm.tqdm(glob.glob(calib_dir+'/*.corners.p')):
    
    prefix = path.split('.corners')[0]
    uvs,ixs = map(np.array,joblib.load(path))
    markers = joblib.load(prefix+'.circle_plus.p')['uv']
    uvs = orient_board(uvs, markers[ixs], board=board)
    chessboard_uvs = np.zeros((18001,70,2))*np.nan
    chessboard_uvs[ixs] = uvs

    all_chessboard_uvs.append(chessboard_uvs)
    prefixes.append(prefix)

all_calib_uvs = np.array(all_chessboard_uvs)

img_sizes = [
    (1280, 1024),
    (1280, 1024),
    (1280, 832),
    (1280, 832),
    (1280, 832),
    (1280, 832)]

# Initial calibration

In [ ]:
from multicam_calibration import calibrate, plot_residuals, chessboard

In [ ]:
chess_board_shape = (7,10)
chess_board_square_size = 12.5
calib_objpoints = chessboard.generate_objpoints(chess_board_shape, chess_board_square_size)

all_extrinsics, all_intrinsics, calib_poses = calibrate(
    all_calib_uvs, img_sizes, calib_objpoints, root=0)


In [ ]:
fig, mean_squared_error, reprojections, transformed_reprojections = plot_residuals(
    all_calib_uvs, all_extrinsics, all_intrinsics, calib_objpoints, calib_poses)

# Bundle Adjustment

In [ ]:
adj_extrinsics, adj_intrinsics, adj_calib_poses, use_frames, result = \
    bundle_adjust(all_calib_uvs, all_extrinsics, all_intrinsics, calib_poses, calib_objpoints, n_frames=500, ftol=1e-4)

In [ ]:
fig, median_error, reprojections, transformed_reprojections = plot_residuals(
    all_calib_uvs[:,use_frames], adj_extrinsics, adj_intrinsics, adj_calib_positions, calib_objpoints)